# Transalate Kaggle data into Source table

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import functions as F
from snowflake.snowpark.window import Window
session = get_active_session()


df_amazon = session.table("hulu_MOVIES_TV")
df_disney = session.table("disney_MOVIES_TV")
df_hulu = session.table("HULU_MOVIES_TV")
df_netflix = session.table("netflix_MOVIES_TV")

df_industry = session.table("INDUSTRY_MOVIES")
df_IMDB = session.table("IMDB_MOVIE_REVIEWS")



# Select only the required columns for df_amazon
df_amazon_with_columns = df_amazon \
    .select("TYPE", "TITLE", "DIRECTOR", "CAST", "COUNTRY", "RATING", "DATE_ADDED", 
            "RELEASE_YEAR", "DURATION", "LISTED_IN")

# Select only the required columns for df_disney
df_disney_with_columns = df_disney \
    .select("TYPE", "TITLE", "DIRECTOR", "CAST", "COUNTRY", "RATING", "DATE_ADDED", 
            "RELEASE_YEAR", "DURATION", "LISTED_IN")

# Select only the required columns for df_hulu
df_hulu_with_columns = df_hulu \
    .select("TYPE", "TITLE", "DIRECTOR", "CAST", "COUNTRY", "RATING", "DATE_ADDED", 
            "RELEASE_YEAR", "DURATION", "LISTED_IN")

# Select only the required columns for df_netflix
df_netflix_with_columns = df_netflix \
    .select("TYPE", "TITLE", "DIRECTOR", "CAST", "COUNTRY", "RATING", "DATE_ADDED", 
            "RELEASE_YEAR", "DURATION", "LISTED_IN")

# Union all the DataFrames into df_movie with the required columns
df_movie = df_amazon_with_columns \
    .union(df_disney_with_columns) \
    .union(df_hulu_with_columns) \
    .union(df_netflix_with_columns)

# Drop duplicate titles
df_movie = df_movie.groupBy("TITLE").agg(
    F.coalesce(F.max("TYPE"), F.lit(None)).alias("TYPE"),
    F.coalesce(F.max("DIRECTOR"), F.lit(None)).alias("DIRECTOR"),
    F.coalesce(F.max("CAST"), F.lit(None)).alias("CAST"),
    F.coalesce(F.max("COUNTRY"), F.lit(None)).alias("COUNTRY"),
    F.coalesce(F.max("RATING"), F.lit(None)).alias("RATING"),
    F.coalesce(F.max("DATE_ADDED"), F.lit(None)).alias("DATE_ADDED"),
    F.coalesce(F.max("RELEASE_YEAR"), F.lit(None)).alias("RELEASE_YEAR"),
    F.coalesce(F.max("DURATION"), F.lit(None)).alias("DURATION"),
    F.coalesce(F.max("LISTED_IN"), F.lit(None)).alias("LISTED_IN")
)

# Add primary key (row number)
df_movie = df_movie.withColumn("Movies_key", F.row_number().over(Window.orderBy(F.lit(1))))

# Save to table (assuming Snowflake environment is set up)
df_movie.write \
    .mode("overwrite") \
    .saveAsTable("MOVIES_TV")


### adding streaming tags
# Add 'amazon_rating' and 'amazon' flag
df_amazon_with_rating = df_amazon \
    .select("TITLE") \
    .with_column("amazon", F.lit(True))

# Add 'disney_rating' and 'disney' flag
df_disney_with_rating = df_disney \
    .select("TITLE") \
    .with_column("disney", F.lit(True))

# Add 'hulu_rating' and 'hulu' flag
df_hulu_with_rating = df_hulu \
    .select("TITLE") \
    .with_column("hulu", F.lit(True))

# Add 'netflix_rating' and 'netflix' flag
df_netflix_with_rating = df_netflix \
    .select("TITLE") \
    .with_column("netflix", F.lit(True))

# Perform left joins with df_movie
df_movie = df_movie \
    .join(df_amazon_with_rating, on="TITLE", how="left") \
    .join(df_disney_with_rating, on="TITLE", how="left") \
    .join(df_hulu_with_rating, on="TITLE", how="left") \
    .join(df_netflix_with_rating, on="TITLE", how="left")


# Save to table (assuming Snowflake environment is set up)
df_movie.write \
    .mode("overwrite") \
    .save_as_table("MOVIES_TV")

# join iMBD TABLE and INDUSTRY Table 
df_industry = df_industry.select(
    df_industry["NAME"].alias("TITLE"),
    df_industry["SCORE"].alias("INDUSTRY_SCORE"),
    df_industry["VOTES"].alias("INDUSTRY_VOTES"),
    df_industry["Budget"],
    df_industry["GROSS"],
    df_industry["COMPANY"]
)

df_IMDB = df_IMDB.select(
    df_IMDB["NAME"].alias("TITLE"),
    df_IMDB["IMBD_ID"].alias("IMBD"),
    df_IMDB["RATING"].alias("IMBD_RATING"),
    df_IMDB["METASCORE"],
    df_IMDB["VOTES"].alias("IMBD_VOTES")
)

# Perform a left join on the "TITLE" column
df_INDUSTRY_JOIN = df_industry.join(df_IMDB, on="TITLE", how="left")

# Save to table (assuming Snowflake environment is set up)
df_INDUSTRY_JOIN.write \
    .mode("overwrite") \
    .save_as_table("MOVIES_TV_REVIEWS")

# Perform a left join on the "TITLE" column
df_streaming = df_movie.join(df_INDUSTRY_JOIN, on="TITLE", how="left")
df_streaming.show()

# Save to table (assuming Snowflake environment is set up)
df_streaming.write \
    .mode("overwrite") \
    .save_as_table("STREAMING_MOVIES_TV")


# Creating normative table from STREAMING_MOVIES_TV


## Entities

### 1. **nf_movies_tv_shows**
| **Field**       | **Type**  | **Description**                                        |
|-----------------|-----------|--------------------------------------------------------|
| MOVIES_KEY      | INT       | Primary Key. Unique identifier for each movie/show.    |
| TITLE           | VARCHAR   | Title of the movie or TV show.                         |
| TYPE            | VARCHAR   | Type of content (e.g., Movie, TV Show).                |
| RATING          | VARCHAR   | Rating (e.g., TV-PG, R, NULL for some).                |
| DATE_ADDED      | DATE      | Date when the movie/show was added.                    |
| RELEASE_YEAR    | INT       | Year of release.                                       |
| DURATION        | VARCHAR   | Duration of the movie/show (e.g., 88 min, 1 Season).   |
| LISTED_IN       | VARCHAR   | Categories of the movie/show (e.g., Horror, Thriller). |

### 2. **nf_cast**
| **Field**       | **Type**  | **Description**                                        |
|-----------------|-----------|--------------------------------------------------------|
| MOVIES_KEY      | INT       | Foreign Key. References MOVIES_KEY in `nf_movies_tv_shows`. |
| nf_cast            | VARCHAR   | Name(s) of the nf_cast members. (NULL if unavailable).    |

### 3. **nf_countries**
| **Field**       | **Type**  | **Description**                                        |
|-----------------|-----------|--------------------------------------------------------|
| MOVIES_KEY      | INT       | Foreign Key. References MOVIES_KEY in `nf_movies_tv_shows`. |
| COUNTRY         | VARCHAR   | Country/Region where the movie/show is available.      |

### 4. **nf_platforms**
| **Field**       | **Type**  | **Description**                                        |
|-----------------|-----------|--------------------------------------------------------|
| MOVIES_KEY      | INT       | Foreign Key. References MOVIES_KEY in `nf_movies_tv_shows`. |
| AMAZON          | BOOLEAN   | Availability on Amazon (True/False).                   |
| DISNEY          | BOOLEAN   | Availability on Disney+ (True/False).                  |
| HULU            | BOOLEAN   | Availability on Hulu (True/False).                     |
| NETFLIX         | BOOLEAN   | Availability on Netflix (True/False).                  |

### 5. **nf_ratings**
| **Field**       | **Type**  | **Description**                                        |
|-----------------|-----------|--------------------------------------------------------|
| MOVIES_KEY      | INT       | Foreign Key. References MOVIES_KEY in `nf_movies_tv_shows`. |
| IMBD            | INT       | IMDB ID for the movie/show.                            |
| IMBD_RATING     | FLOAT     | IMDB rating of the movie/show.                         |
| METASCORE       | INT       | Metascore rating of the movie/show.                    |
| IMBD_VOTES      | INT       | Number of IMDB votes for the movie/show.               |

### 6. **nf_industry**
| **Field**       | **Type**  | **Description**                                        |
|-----------------|-----------|--------------------------------------------------------|
| MOVIES_KEY      | INT       | Foreign Key. References MOVIES_KEY in `nf_movies_tv_shows`. |
| BUDGET          | DECIMAL   | Budget of the movie/show.                              |
| GROSS           | DECIMAL   | Gross earnings of the movie/show.                      |
| COMPANY         | VARCHAR   | Production company of the movie/show.                  |



In [ ]:
CREATE OR REPLACE TABLE nf_movies_tv_shows AS
SELECT 
    MOVIES_KEY, 
    TITLE, 
    TYPE, 
    RATING, 
    DATE_ADDED, 
    RELEASE_YEAR, 
    DURATION, 
    LISTED_IN
FROM STREAMING_MOVIES_TV;
ALTER TABLE nf_movies_tv_shows
ADD PRIMARY KEY (MOVIES_KEY);


CREATE OR REPLACE TABLE nf_cast AS
SELECT 
    MOVIES_KEY, 
    "CAST"
FROM STREAMING_MOVIES_TV
WHERE "CAST" IS NOT NULL;
ALTER TABLE nf_countries
ADD CONSTRAINT fk_movies_key FOREIGN KEY (MOVIES_KEY) REFERENCES nf_movies_tv_shows(MOVIES_KEY);


CREATE OR REPLACE TABLE nf_countries AS
SELECT 
    MOVIES_KEY, 
    COUNTRY
FROM STREAMING_MOVIES_TV
WHERE COUNTRY IS NOT NULL;
ALTER TABLE nf_countries
ADD CONSTRAINT fk_movies_key FOREIGN KEY (MOVIES_KEY) REFERENCES nf_movies_tv_shows(MOVIES_KEY);


CREATE OR REPLACE TABLE nf_platforms AS
SELECT 
    MOVIES_KEY, 
    amazon, 
    disney, 
    hulu, 
    netflix
FROM STREAMING_MOVIES_TV;
ALTER TABLE nf_platforms
ADD CONSTRAINT fk_movies_key FOREIGN KEY (MOVIES_KEY) REFERENCES nf_movies_tv_shows(MOVIES_KEY);


CREATE OR REPLACE TABLE nf_ratings AS
SELECT 
    MOVIES_KEY, 
    IMBD, 
    IMBD_RATING, 
    METASCORE, 
    IMBD_VOTES
FROM STREAMING_MOVIES_TV;
ALTER TABLE nf_ratings
ADD CONSTRAINT fk_movies_key FOREIGN KEY (MOVIES_KEY) REFERENCES nf_movies_tv_shows(MOVIES_KEY);


CREATE OR REPLACE TABLE nf_industry AS
SELECT 
    MOVIES_KEY, 
    BUDGET, 
    GROSS, 
    COMPANY
FROM STREAMING_MOVIES_TV;
ALTER TABLE nf_industry
ADD CONSTRAINT fk_movies_key FOREIGN KEY (MOVIES_KEY) REFERENCES nf_movies_tv_shows(MOVIES_KEY);